In [57]:
import tkinter as tk
from tkinter import Tk, ttk, filedialog
from PIL import Image, ImageTk, ImageOps
from tkVideoPlayer import TkinterVideo

import cv2
import numpy as np
import sys

from datetime import date, datetime, time, timedelta

# helper python code
import timeParse # extract time data from frames

# timers for efficiency testing
from functools import wraps
import time

def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__}{args} {kwargs} took {total_time:.4f} seconds')
        return result
    return timeit_wrapper

In [56]:
class VideoFeed(tk.Frame):
    def __init__(self, file_path=None, init_index=0, time_chunk=5, fps=None, parent=None):
        super().__init__()
        root = Tk() if parent == None else parent # pipe outwards
        self.parent = root
        
        
        self.parent.rowconfigure(0,weight=3)
        self.parent.columnconfigure(0,weight=3)
        
        # video setup
        self.file_path = file_path
        
        self.vidProperty(init_index=init_index, fps=fps, time_chunk=time_chunk)
        self.customTk()
#         self.tkVideo()
        
#         if self.file_path:
#             self.vid_player.load(self.file_path)

#             self.progress_slider.config(to=0, from_=0)
#             self.play_pause_btn["text"] = "Play"
#             self.progress_value.set(0)
            
#             self.vidProperty(init_index=init_index, fps=fps, time_chunk=time_chunk)
            
#             self.play_pause()
#             self.quickPlayed = True
#             self.updateVideoFrame()
        
        # test
#         self.jumpForwardTime(timedelta(minutes=3))
#         self.showFrame(self.frame, show=True)
        
#         # display
#         self.parent.update()
#         print(self.parent)
#         self.displayFrame()

    def vidProperty(self, init_index=None, fps=None, time_chunk=None):
        self.cap = self.getCapture(self.file_path)
        print("created capture")
        self.dims = self.cap.read()[1].shape
        
        # number of frames
        self.frame_count = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print("found framecount")
        
        # fps (camera)
        self.fps = self.getFPS(self.cap) if fps == None else fps 
        print("found fps")
        
        # fps/framerate (video)
#         self.framerate = self.vid_player.video_info()['framerate']
#         print("found framerate")
        
        # time in camera time
        self.time_chunk = timedelta(seconds=time_chunk)
        print("set properties")
        
        # initiate variables
        self.frame_index = init_index
        self.frame = self.getFrame(self.frame_index)
        print("set variables")
        
        self.startTime = timeParse.getDateTime(self.frame)
    
    def updateFrame(self):
        self.frame = self.getFrame(self.frame_index)
        
    def displayFrame(self):
        pass
    
    def resize_image(self, event):
        print("resize triggered")
        new_width = event.width
        new_height = event.height
        
        ratio = (self.dims[0] / self.dims[1])
        
#         if (new_height / new_width) < ratio:
#             new_height = int(new_width * ratio)
#         else:
#             new_width = int(new_height / ratio)

        self.resized_image = ImageOps.contain(self.img.copy(), (new_width, new_height))
        self.resized_image.resize((new_width, new_height))
        self.resized_tk = ImageTk.PhotoImage(self.resized_image)
        self.imgLabel.config(image=self.resized_tk)
    
    def updateDisplay(self):
        self.updateFrame()
        self.displayFrame()
        
    def frameIndexToSecond(self,frame_index):
        return frame_index / self.framerate 
        
    '''
    FRAME ACTION
    '''
    def jumpArbitraryFrame(self, frame_index):
        self.frame_index = frame_index
        self.updateFrame()
    
    def jumpForwardFrames(self, frames_forward):
        self.frame_index = self.frame_index + frames_forward
        self.updateFrame()
    
    def jumpBackFrames(self, frames_back):
        self.frame_index = self.frame_index - frames_back
        self.updateFrame()
    
    def jumpForwardTime(self, delt):
        self.frame_index = self.getFrameIndexOfTime(timeParse.getDateTime(self.frame) + delt)
        self.updateFrame()
    
    def jumpForwardChunk(self):
        self.jumpForwardTime(time_chunk)
        self.updateFrame()
    
    def jumpBackChunk(self):
        self.frame_index = self.getFrameIndexOfTime(timeParse.getDateTime(self.frame) - delt)
        self.updateFrame()
        
    
    '''
    GENERAL SETUP
    '''
    def getCapture(self, filepath):
        return cv2.VideoCapture(filepath)

    def getFrame(self, frame_index, cap=None, show=False):
        """Retrieves the frame from the video

        Retrieve the indexed frame from the video and return it as an image of an numpy array.

        Args:
            frame_num: index of the frame to show.
            cap: capture to find the frame of, defaults to cap defined above
            show: determines whether to show or to skip, defaults to noshow

        Returns:
            numpy image array of the frame

        Raises:
            Exception: frame_num({frame_num}) cannot equal or exceed number of frames({frame_count})
        """
        cap = self.cap if cap == None else cap

        if (frame_index >= self.frame_count): 
            raise Exception(f"frame_index({frame_index}) cannot equal or exceed number of frames({self.frame_count})") 

        self.cap.set(1, frame_index)  # where frame_no is the frame you want
        ret, frame = cap.read()  # read the frame

        self.showFrame(frame, show, title="frame")

        return frame

    @timeit
    def getFPS(self, cap=None, n=3):
        cap = self.cap if cap == None else cap
        
        init_time = timeParse.getTime(self.getFrame(0))
        
        j = 0
        while j < self.frame_count and timeParse.getTime(self.getFrame(j)) == init_time: j += 1

        secCounter = 0
        i = j
        
         # start of the first sec
        new_time = timeParse.getTime(self.getFrame(j))
        while secCounter < n:
            while i < self.frame_count and timeParse.getTime(self.getFrame(i)) == new_time: i += 1 # start of the second second
            
            new_time = timeParse.getTime(self.getFrame(i))
            secCounter += 1

        return (i - j) / n

    @timeit
    def getAccurateIndex(self, index, time, fps=None, cap=None):
        fps = self.fps if fps == None else fps
        cap = self.cap if cap == None else cap
        
        while (timeParse.getDateTime(self.getFrame(index)) >= time):
            index -= 1

        while (timeParse.getDateTime(self.getFrame(index)) < time):
            index += 1
            
        return index

    def getFrameIndexOfTime(self, time, fps=None, cap=None):
        fps = self.fps if fps == None else fps
        cap = self.cap if cap == None else cap
        
        init_time = timeParse.getDateTime(self.getFrame(0))
        time_delt = time - init_time

        if (time_delt.total_seconds() < 0): return -1

        jump_index = time_delt.total_seconds() * fps
        return self.getAccurateIndex(jump_index, time)
    
    @staticmethod
    def showFrame(frame, show, title="Unnamed Frame"):
        """Show the frame in question if show is true for debugging.

        If show is false, does nothing.
        If show is true, shows frame as an external window until a keypress is recorded.

        Args:
            frame: frame to show.
            title: title of the frame to show. Defaults to "Unnamed Frame"
            show: determines whether to show or to skip

        Returns:
            Nothing

        Raises:
        """

        if (show):
            cv2.imshow(title, frame)  # show frame on window
            cv2.waitKey(0)

            # closing all open windows
            cv2.destroyAllWindows()
            
    '''
    TkVideo Setup
    '''
    def update_duration(self, event):
        """ updates the duration after finding the duration """
        self.duration = self.vid_player.video_info()["duration"]
        self.end_time["text"] = str(timedelta(seconds=self.duration))
        self.progress_slider["to"] = self.duration
        
#         self.updateVideoFrame()


    def update_scale(self, event):
        """ updates the scale value """
        self.progress_value.set(self.vid_player.current_duration())


    def load_video(self):
        """ loads the video """
        self.file_path = filedialog.askopenfilename()

        if self.file_path:
            self.vid_player.load(self.file_path)

            self.progress_slider.config(to=0, from_=0)
            self.play_pause_btn["text"] = "Play"
            self.progress_value.set(0)
            
            self.vidProperty()


    def seek(self, value):
        """ used to seek a specific timeframe """
        self.vid_player.seek(int(value))


    def skip(self, value: int):
        """ skip seconds """
        self.vid_player.seek(int(self.progress_slider.get())+value)
        self.progress_value.set(self.progress_slider.get() + value)
        
    def jump(self, value: int):
        """ jump to second """
        self.vid_player.seek(value)
        self.progress_value.set(value)


    def play_pause(self):
        """ pauses and plays """
        if self.vid_player.is_paused():
            self.vid_player.play()
            self.play_pause_btn["text"] = "Pause"

        else:
            self.vid_player.pause()
            self.play_pause_btn["text"] = "Play"
        
    def updateVideoFrame(self):
        if self.quickPlayed == False:
            return
            
        self.play_pause()
        self.quickPlayed = False

    def video_ended(self, event):
        """ handle video ended """
        self.progress_slider.set(self.progress_slider["to"])
        self.play_pause_btn["text"] = "Play"
        self.progress_slider.set(0)
    
    def customTk(self):
#         imgTk = ImageTk.PhotoImage(Image.fromarray(self.frame).resize((self.parent.winfo_width(), 
#                                                                      self.parent.winfo_height())), master=self.parent)
#         self.vid_player = tk.Label(image=imageTk, scaled=True, master=self.parent)
#         self.vid_player.grid(row=0,column=0, sticky="nsew", columnspan=10)
        self.updateFrame()
#         self.showFrame(self.frame, show=True)
        self.img = Image.fromarray(self.frame)
        self.imgTk = ImageTk.PhotoImage(self.img, master=self.parent)
        self.imgLabel = tk.Label(self.parent, image=self.imgTk, width=700, height=700*int(self.dims[0] / self.dims[1]))
        self.imgLabel.grid(row=0, column=0, columnspan=10, sticky="new")
        self.imgLabel.bind("<Configure>", self.resize_image)
    
    
    def tkVideo(self):
        
        '''
        GUI Setup
        '''
        self.load_btn = tk.Button(self.parent, text="Load", command=self.load_video)
        self.load_btn.pack()

        self.vid_player = TkinterVideo(scaled=True, master=self.parent)
#         self.vid_player.grid(row=0,column=0, rowspan=10, columnspan=10)
        self.vid_player.pack(expand=True, fill="both")

        self.play_pause_btn = tk.Button(self.parent, text="Play", command=self.play_pause)
        self.play_pause_btn.pack()
#         self.play_pause_btn.grid(row=8,column=4, rowspan=1)
        
#         self.update_pause_btn = tk.Button(self.parent, text="Update", command=self.updateVideoFrame)
#         self.update_pause_btn.pack(side="top", anchor="n")
#         self.update_pause_btn.grid(row=8,column=5, rowspan=1)

        self.skip_plus_5sec = tk.Button(self.parent, text="Skip -1 sec", command=lambda: self.skip(-1))
        self.skip_plus_5sec.pack(side="left")
#         self.skip_plus_5sec.grid(row=9,column=0, sticky="w")

        self.start_time = tk.Label(self.parent, text=str(timedelta(seconds=0)))
        self.start_time.pack(side="left")
#         self.start_time.grid(row=9,column=1, rowspan=1, sticky="w")

        self.progress_value = tk.IntVar(self.parent)

        self.progress_slider = tk.Scale(self.parent, 
                                   variable=self.progress_value, 
                                   from_=0, to=0, 
                                   orient="horizontal", 
                                   command=self.seek)
        
#         progress_slider.bind("<ButtonRelease-1>", seek)
        self.progress_slider.pack(side="left", fill="x", expand=True)
#         self.progress_slider.grid(row=9,column=2, columnspan=10, sticky="nsew")

        self.end_time = tk.Label(self.parent, text=str(timedelta(seconds=0)))
        self.end_time.pack(side="left")
#         self.end_time.grid(row=9,column=9, rowspan=1, sticky="e")

        self.vid_player.bind("<<Duration>>", self.update_duration)
        self.vid_player.bind("<<SecondChanged>>", self.update_scale)
        self.vid_player.bind("<<Ended>>", self.video_ended )

        self.skip_plus_5sec = tk.Button(self.parent, text="Skip +1 sec", command=lambda: self.skip(1))
        self.skip_plus_5sec.pack(side="left")
#         self.skip_plus_5sec.grid(row=9,column=9, rowspan=1, sticky="e")
        
        self.vid_player.keep_aspect(True)
        


if __name__ == "__main__":
    root = Tk()
    vf = VideoFeed(file_path="vids\TLC00011.mp4", parent=root)
#     vf = VideoFeed("vids\TLC00011.mp4", parent=root, fps=5)
    print("created object")
    root.mainloop()

created capture
found framecount
Function getFPS(<__main__.VideoFeed object .!videofeed>, <VideoCapture 000002B968B24FB0>) {} took 1.8349 seconds
found fps
set properties
set variables
created object
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
resize triggered
r

In [39]:
vf.ratio

(720, 1280, 3)

In [6]:
import tkinter as tk
from PIL import Image, ImageTk, ImageOps

root = tk.Tk()
canvas = tk.Canvas(root, width=500, height=500, bg="blue")
canvas.grid(row=0, column=0, sticky="nsew")
root.rowconfigure(0,weight=1)
root.columnconfigure(0,weight=1)

imgBottom = Image.open('../vids/singleframe.png')
imgBottomtk = ImageTk.PhotoImage(imgBottom)
imageBottom = canvas.create_image(imgBottom.size[0] // 2, imgBottom.size[1] // 2, image=imgBottomtk)

l = []

# vid_frame_dim =  300, 200
# #     resized_image = ImageOps.contain(imgBottom.copy(), (vid_frame_dim[0], vid_frame_dim[1]))
# resized_image = ImageOps.contain(imgBottom.copy(), (300, 200))
# resized_image.resize((300, 200))
# resized_tk = ImageTk.PhotoImage(resized_image)

# canvas.itemconfig(imageBottom, image=resized_tk)
# canvas.coords(imageBottom, resized_image.size[0] // 2, resized_image.size[1] // 2)

def resize():
    vid_frame_dim =  300, 200
#     resized_image = ImageOps.contain(imgBottom.copy(), (vid_frame_dim[0], vid_frame_dim[1]))
    resized_image = ImageOps.contain(imgBottom.copy(), (300, 200))
    resized_image.resize((300, 200))
    resized_tk = ImageTk.PhotoImage(resized_image)
    l.append(resized_tk)

    canvas.itemconfig(imageBottom, image=resized_tk)
    canvas.coords(imageBottom, resized_image.size[0] // 2, resized_image.size[1] // 2)
    
resize()

root.mainloop()

In [7]:
import tkinter as tk
from PIL import Image, ImageTk


root = tk.Tk()
canvas = tk.Canvas(root, width=500, height=500)
canvas.grid(row=0, column=0, sticky="nsew")
root.rowconfigure(0,weight=1)
root.columnconfigure(0,weight=1)

imgBottom = Image.open('../vids/singleframe.png')
imgBottomtk = ImageTk.PhotoImage(imgBottom)
imageBottom = canvas.create_image(imgBottom.size[0] // 2, imgBottom.size[1] // 2, image=imgBottomtk)

# imgOverlay = Image.open('overlays/TLC00011.png').convert(mode="RGBA")
# imgtk = ImageTk.PhotoImage(imgOverlay)
# image = canvas.create_image(imgBottom.size[0] // 2, imgBottom.size[1] // 2, image=imgtk)


# newImgOverlay = Image.open('overlays/beeg.png').convert(mode="RGBA")
# newImgtk = ImageTk.PhotoImage(newImgOverlay)
# canvas.itemconfig(image, image=newImgtk)

l = 

def _resize_image(event):
    print("resize triggered")
    vid_frame_dim = event.width, event.height
    print(vid_frame_dim)

    # resize image
    resized_image = ImageOps.contain(imgBottom.copy(), (vid_frame_dim[0], vid_frame_dim[1]))
    resized_image.resize((vid_frame_dim[0], vid_frame_dim[1]))
    resized_tk = ImageTk.PhotoImage(resized_image)
    
    print(resized_image.size[0] // 2, resized_image.size[1] // 2)
    canvas.itemconfig(imageBottom, image=resized_tk)
    canvas.coords(imageBottom, resized_image.size[0] // 2, resized_image.size[1] // 2)

#     resized_overlay = ImageOps.contain(newImgOverlay.copy(), (vid_frame_dim[0], vid_frame_dim[1]))
#     resized_overlay.resize((vid_frame_dim[0], vid_frame_dim[1]))
# #     canvas.image = resized_overlay
#     resized_overlay_tk = ImageTk.PhotoImage(resized_overlay.convert(mode="RGBA"))
#     canvas.itemconfig(image, image=resized_overlay_tk)

root.bind("<Configure>", _resize_image)
    
root.mainloop()

resize triggered
(504, 504)
252 141
resize triggered
(504, 504)
252 141
resize triggered
(513, 504)
256 144
resize triggered
(513, 504)
256 144
resize triggered
(597, 504)
298 167
resize triggered
(597, 504)
298 167
resize triggered
(838, 504)
419 235
resize triggered
(838, 504)
419 235
resize triggered
(1052, 504)
448 252
resize triggered
(1052, 504)
448 252
resize triggered
(1091, 504)
448 252
resize triggered
(1091, 504)
448 252
resize triggered
(1107, 504)
448 252
resize triggered
(1107, 504)
448 252
resize triggered
(1113, 504)
448 252
resize triggered
(1113, 504)
448 252
resize triggered
(1113, 504)
448 252
resize triggered
(1113, 504)
448 252
resize triggered
(1113, 504)
448 252
resize triggered
(1113, 504)
448 252
resize triggered
(1113, 504)
448 252
resize triggered
(1113, 504)
448 252
resize triggered
(1113, 504)
448 252
resize triggered
(1113, 504)
448 252
resize triggered
(1113, 503)
447 251
resize triggered
(1113, 503)
447 251
resize triggered
(1113, 511)
454 255
resize tr